# Load Packages

In [5]:
%load_ext autoreload
%autoreload 2

import sys
from os.path import join
from tqdm.auto import tqdm
import joblib
import torch
sys.path.append("../../")

from src.evaluation.consolidate import consolidate_pred_perf, consolidate_ue_perf, consolidate_pi_perf
from src.evaluation.perf_eval import display_pred_perf
from src.evaluation.ue_eval import display_ue_perf, restructure_ue_df
from src.evaluation.pi_eval import display_pi_perf
from src.df_display.latex import df_to_latex, df_to_latex_grouped
from src.misc import create_folder

seed_list=list(range(2023, 2023+5))
data_label = "physionet"
batch_size = 64

# File paths
fp_notebooks_folder = "../"
fp_project_folder = join(fp_notebooks_folder, "../")
fp_data_folder = join(fp_project_folder, "../", "data")
fp_output_data_folder = join(fp_data_folder, data_label)
fp_checkpoint_folder = join(fp_project_folder, "checkpoints")
fp_project_checkpoints = join(fp_checkpoint_folder, data_label)
fp_tuning = join(fp_project_checkpoints, "tuning")
fp_models = join(fp_project_checkpoints, "models")
fp_predictions = join(fp_project_checkpoints, "predictions")
fp_evaluation = join(fp_project_checkpoints, "model_evaluation")
fp_consolidated = join(fp_project_checkpoints, "consolidated_results")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Prediction Performance

In [2]:
pred_perf_df = consolidate_pred_perf(seed_list, fp_evaluation)
display_pred_perf(pred_perf_df, consolidated=True)
pred_perf_df.to_csv(join(fp_consolidated, "pred_perf.csv"))

,t+1,t+2,t+3
Model,,,
RUE,0.002 ± 0.000,0.0024 ± 0.000,0.0027 ± 0.000
MC Dropout,0.002 ± 0.000,0.0024 ± 0.000,0.0027 ± 0.000
GPR,0.0019 ± 0.000,0.0024 ± 0.000,0.0027 ± 0.000
Infer-Noise,0.002 ± 0.000,0.0024 ± 0.000,0.0027 ± 0.000
BNN,0.0052 ± 0.000,0.0053 ± 0.000,0.0053 ± 0.000
DER,0.0021 ± 0.000,0.0025 ± 0.000,0.003 ± 0.000


In [3]:
print(df_to_latex(pred_perf_df, column_format_dict={"t+1": "min", "t+2": "min", "t+3": "min"}))

\begin{tabular}{cccc}
 & \textbf{t+1} & \textbf{t+2} & \textbf{t+3} \\
Model &  &  &  \\
RUE & \underline{0.002 ± 0.000} & \textbf{0.0024 ± 0.000} & \textbf{0.0027 ± 0.000} \\
MC Dropout & \underline{0.002 ± 0.000} & \textbf{0.0024 ± 0.000} & \textbf{0.0027 ± 0.000} \\
GPR & \textbf{0.0019 ± 0.000} & \textbf{0.0024 ± 0.000} & \textbf{0.0027 ± 0.000} \\
Infer-Noise & \underline{0.002 ± 0.000} & \textbf{0.0024 ± 0.000} & \textbf{0.0027 ± 0.000} \\
BNN & 0.0052 ± 0.000 & 0.0053 ± 0.000 & 0.0053 ± 0.000 \\
DER & 0.0021 ± 0.000 & \underline{0.0025 ± 0.000} & \underline{0.003 ± 0.000} \\
\end{tabular}



# UE Performance

In [17]:
ue_perf_df = consolidate_ue_perf(seed_list, fp_evaluation, exclude_columns="Pval")
display_ue_perf(ue_perf_df, consolidated=True)

t+1


t+2


t+3


In [18]:
print(
    df_to_latex_grouped(
        ue_perf_df, 
        {"Corr": "max", "AURC":"min", "Sigma=0.1": "min", "Sigma=0.2": "min", "Sigma=0.3": "min", "Sigma=0.4": "min"})
)

\begin{tabular}{cccccccc}
\toprule
 &  & Corr & AURC & Sigma=0.1 & Sigma=0.2 & Sigma=0.3 & Sigma=0.4 \\
Time Horizon & Model &  &  &  &  &  &  \\
\midrule
\multirow[t]{6}{*}{t+1} & RUE & \textbf{0.282 ± 0.014} & \underline{0.025 ± 0.001} & \textbf{0.02 ± 0.002} & \textbf{0.022 ± 0.001} & \textbf{0.024 ± 0.001} & \textbf{0.025 ± 0.001} \\
 & MC Dropout & 0.189 ± 0.012 & 0.026 ± 0.001 & 0.024 ± 0.001 & 0.025 ± 0.001 & 0.026 ± 0.001 & \underline{0.026 ± 0.001} \\
 & GPR & 0.174 ± 0.001 & \textbf{0.024 ± 0.000} & \underline{0.022 ± 0.000} & 0.024 ± 0.000 & \underline{0.025 ± 0.000} & \textbf{0.025 ± 0.000} \\
 & Infer-Noise & 0.057 ± 0.012 & 0.028 ± 0.001 & 0.028 ± 0.003 & 0.027 ± 0.001 & 0.028 ± 0.001 & 0.028 ± 0.001 \\
 & BNN & 0.122 ± 0.004 & 0.05 ± 0.000 & 0.049 ± 0.003 & 0.048 ± 0.001 & 0.049 ± 0.002 & 0.05 ± 0.001 \\
 & DER & \underline{0.236 ± 0.013} & \underline{0.025 ± 0.001} & \underline{0.022 ± 0.001} & \underline{0.023 ± 0.001} & \underline{0.025 ± 0.001} & \underline{0.026 ± 0

# PI Performance

In [29]:
pi_perf_df = consolidate_pi_perf(seed_list, fp_evaluation, exclude_columns=["PICP"])
display_pi_perf(pi_perf_df, consolidated=True)
pi_perf_df.to_csv(join(fp_consolidated, "pi_perf.csv"))

t+1:


t+2:


t+3:


In [27]:
print(
    df_to_latex_grouped(
        pi_perf_df, 
        {"PINAW":"min", "PINAFD": "min", "CovP":"min", "CWFDC":"min"})
)

\begin{tabular}{cccccc}
\toprule
 &  & PINAW & PINAFD & CovP & CWFDC \\
Time Horizon & Method &  &  &  &  \\
\midrule
\multirow[t]{10}{*}{t+1} & RUE Gaussian Copula & \underline{0.22075 ± 0.002} & 0.05407 ± 0.001 & \underline{8e-05 ± 0.000} & 0.35331 ± 0.011 \\
 & RUE Conditional Gaussian & \textbf{0.20544 ± 0.001} & 0.05293 ± 0.001 & 9e-05 ± 0.000 & \underline{0.34739 ± 0.014} \\
 & RUE Weighted & 0.30154 ± 0.006 & 0.05351 ± 0.001 & 0.00028 ± 0.000 & 0.63474 ± 0.030 \\
 & RUE KNN & 0.84647 ± 0.011 & 0.10399 ± 0.007 & 0.00232 ± 0.000 & 3.27213 ± 0.017 \\
 & RUE Conformal & 0.2346 ± 0.004 & \textbf{0.04919 ± 0.001} & \textbf{5e-05 ± 0.000} & \textbf{0.33146 ± 0.014} \\
 & Infer-Noise Conformal & 0.25125 ± 0.004 & 0.05709 ± 0.001 & 0.00024 ± 0.000 & 0.54378 ± 0.033 \\
 & MC Dropout Conformal & 0.27546 ± 0.006 & \underline{0.05157 ± 0.001} & 9e-05 ± 0.000 & 0.41215 ± 0.015 \\
 & GPR Conformal & 0.23211 ± 0.000 & 0.05584 ± 0.000 & 0.00011 ± 0.000 & 0.39299 ± 0.001 \\
 & BNN Conformal & 0.3